## Goal: to create 5 new features: normalize first

    1. KL-divergence between 35 and 350: change hyperparameter 0.1 too high
	2. KL-divergence between zero standard and each sample distritbution
	3. Mode-mean, Mode-medium
	4. R2 vlaue to fit a Gaussian curve(does it look like a Gaussian?)
	5. The count of peaks (Unimodal, Bimodal etc)
	
## Add into the dataset and test with the tree model
## Build a new DNN and try
    

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from sklearn.neighbors import KernelDensity

'''
    df_invL35 = df.iloc[:, 1:36]
    df_invL350 = df.iloc[:, 36:71]
    df_L35 = df.iloc[:, 71:106]
    df_L350 = df.iloc[:, 106:141]
'''
df = pd.read_csv('Raw.csv',header=[0], skiprows=0, index_col=0)
# df['std_invL35'] = df.apply(lambda row: np.std(row[1:36]), axis = 1) 
# # current features df
# feature_df = pd.read_excel('counts.xlsx', sheet_name='Summary Selected', header=[0], skiprows=0, index_col=0)
# feature_df = feature_df[:86]

In [4]:

## Normalize the data!!!
transformed_list = []
raw_values = df.values
for row in raw_values:
    row_list = list(row[1:36]/sum(row[1:36])) +  list(row[36:71]/sum(row[36:71])) + list(row[71:106]/sum(row[71:106])) + list(row[106:141]/sum(row[106:141]))
    transformed_list.append(row_list)
normalized_df = pd.DataFrame(transformed_list)
normalized_df.insert(loc=0, column='Patient', value=df['Patient'].values)
# normalized_df.insert(loc = len(normalized_df.columns.values),column='std_invL35', value=df['std_invL35'].values)
df = normalized_df
print('df.shape: ', df.shape)
# print('feature_df.shape: ', feature_df.shape)
df

df.shape:  (85, 141)


,Patient,0,1,2,3,4,5,6,7,8,...,130,131,132,133,134,135,136,137,138,139
0,-1,0.0,0.0,0.0,0.0,0.0,0.028213,0.028213,0.006270,0.015674,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-1,0.0,0.0,0.0,0.0,0.0,0.003049,0.003049,0.006098,0.000000,...,0.000806,0.001612,0.000806,0.002417,0.000806,0.001612,0.000000,0.000806,0.000000,0.000806
2,-1,0.0,0.0,0.0,0.0,0.0,0.020408,0.008163,0.008163,0.012245,...,0.003175,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,-1,0.0,0.0,0.0,0.0,0.0,0.004975,0.009950,0.012438,0.007463,...,0.000000,0.000000,0.000000,0.001927,0.000000,0.000000,0.000000,0.000000,0.002890,0.000000
4,-1,0.0,0.0,0.0,0.0,0.0,0.000000,0.004202,0.025210,0.008403,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003115,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.013846,0.009231,0.006154,0.006154,0.004615,0.003077,0.000000,0.000000,0.001538,0.000000
81,1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.003484,0.000000,...,0.011013,0.004405,0.004405,0.002203,0.004405,0.002203,0.006608,0.002203,0.000000,0.000000
82,1,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.033649,0.015773,0.018927,0.005258,0.011567,0.007361,0.007361,0.005258,0.003155,0.000000
83,1,0.0,0.0,0.0,0.0,0.0,0.002247,0.013483,0.011236,0.013483,...,0.000000,0.000000,0.002081,0.000000,0.001041,0.000000,0.000000,0.000000,0.000000,0.000000


### 1. KL-divergence between 35 and 350

note: to get rid of zero probability, add epsilon = 1 to every value. (new hyper-parameter!)

In [5]:
import numpy as np

def kl_divergence(p, q):
    """Kullback-Leibler divergence D(P || Q) for discrete distributions
    Parameters: [p, q] : array-like, dtype=float, shape=n, Discrete probability distributions.
    """
    p = np.asarray(p, dtype=np.float)+ 0.001 
    q = np.asarray(q, dtype=np.float)+ 0.001
    return np.sum(np.where(p != 0,  p * np.log(p / q), 0))    
#     return np.sum(np.where(p,q != 0,  p * np.log(p / q), 0))

In [6]:

df['kl_invL'] = df.apply (lambda row: kl_divergence(row[1:36],row[36:71]), axis=1)
df['kl_L'] = df.apply (lambda row: kl_divergence(row[71:106],row[106:141]), axis=1)
df[['kl_invL', 'kl_L']].head()

,kl_invL,kl_L
0,3.125269,3.693250
1,3.860075,2.957620
2,3.113094,3.116387
3,3.970348,2.325744
4,3.067786,3.306703


### 2. KL-divergence from the standard point

note: manaully create a "zero standard"  

In [7]:
from scipy import stats
norm_dis = stats.norm(18, 2)
zero_standard = [norm_dis.pdf(i) for i in range(0,35)]
# print(zero_standard)
# plt.plot(zero_standard)

df['kl_invL35'] = df.apply (lambda row: kl_divergence(row[1:36],zero_standard), axis=1)
df['kl_invL350'] = df.apply (lambda row: kl_divergence(row[36:71],zero_standard), axis=1)
df['kl_L35'] = df.apply (lambda row: kl_divergence(row[71:106],zero_standard), axis=1)
df['kl_L350'] = df.apply (lambda row: kl_divergence(row[106:141],zero_standard), axis=1)
df[['kl_invL35', 'kl_invL350', 'kl_L35', 'kl_L350']]

,kl_invL35,kl_invL350,kl_L35,kl_L350
0,0.436477,4.893190,2.123568,3.835067
1,0.289150,5.025046,1.319488,4.136093
2,0.425664,4.706503,1.308513,4.170995
3,0.255722,4.800706,0.985632,3.941763
4,0.316888,3.980702,0.699152,4.201375
...,...,...,...,...
80,2.392248,3.564432,3.772393,2.642397
81,2.905909,3.661508,3.766270,3.132524
82,2.398793,2.498842,3.992582,1.371058
83,3.078193,4.783823,4.161633,3.870771


### 3. Mode-mean and Mode-medium

question: how to get the Mean_size35 from mean of 35 and mean of Inv35?

In [8]:
# list(feature_df.columns.values)[:5]

In [9]:
def get_mean_index(data):
    mean = 0
    for i, a in enumerate(data):
        mean += i*a
    return mean
    
df['mode_mean_invL35'] = df.apply (lambda row: np.argmax(row[1:36], axis=0) - get_mean_index(row[1:36]), axis=1)
df['mode_mean_invL350'] = df.apply (lambda row: np.argmax(row[36:71], axis=0) - get_mean_index(row[36:71]), axis=1)
df['mode_mean_L35'] = df.apply (lambda row: np.argmax(row[71:106], axis=0) - get_mean_index(row[71:106]), axis=1)
df['mode_mean_L350'] = df.apply (lambda row: np.argmax(row[106:141], axis=0) - get_mean_index(row[106:141]), axis=1)

# df['mode_mean'] = df.apply(lambda row: row['mode_invL35']-row['mean_invL35']+row['mode_invL350']-row['mean_invL350'] +row['mode_L35']-row['mean_L35']+ row['mode_L350']-row['mean_L350'] , axis=1)
# df['mode_mean'].head()

In [11]:
# df['mode_mean_invL350']

### 4. R2 value to fit a Gaussian curve  --> use mean not size


#### Question: how to get the std?? what the hyperparamter shall be? there is no optimal for all sample at the same time
--> just use curve fit

In [12]:
from scipy import optimize
import scipy.stats
from sklearn.metrics import r2_score

def calculate_r2(y_true):
    def gaussian(x, amplitude, mean, stddev):
        return amplitude * np.exp(-((x - mean) / 4 / stddev)**2)

    x = np.linspace(0, 35, 35)
    parameters, _ = optimize.curve_fit(gaussian, x, y_true)    
    y_pred = gaussian(x, *parameters)
    # plt.plot(x, data)
    # plt.plot(x, gaussian(x, *parameters))
    return r2_score(y_true, y_pred)

In [13]:
df['R2_invL35'] = df.apply(lambda row: calculate_r2(row[1:36]), axis=1)
df['R2_invL350'] = df.apply(lambda row: calculate_r2(row[36:71]), axis=1)
df['R2_L35'] = df.apply(lambda row: calculate_r2(row[71:106]), axis=1)
df['R2_L350'] = df.apply(lambda row: calculate_r2(row[106:141]), axis=1)


C:\Anaconda3\envs\heyjody\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)
C:\Anaconda3\envs\heyjody\lib\site-packages\scipy\optimize\minpack.py:808: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


### 5. R2 value Bimodal Gaussian  

https://datascience.stackexchange.com/questions/20397/how-to-model-a-bimodal-distribution-of-target-variable

## convert the df to dataset

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [14]:
new_feature = ['kl_invL','kl_L',
               'kl_invL35','kl_invL350','kl_L35','kl_L350',  
               'mode_mean_invL35','mode_mean_invL350','mode_mean_L35','mode_mean_L350',
               'R2_invL35','R2_invL350','R2_L35', 'R2_L350']
df[new_feature].shape
# df[new_feature].to_csv('14 new features.csv')

(85, 14)

In [15]:
feature_df = pd.read_csv('MK data.csv',encoding = "ISO-8859-1",header=[0], skiprows=0, index_col=0)
feature_df.shape

(85, 29)

In [16]:
feature_df.head()

,Patient,InvL35.Mean,InvL350.Mean,L35.Mean,L350.Mean,Size_InvL35,Size_InvL350,Size_L35,Size_L350,MeanSize_35,...,L35.Skewness,L350.Skewness,InvL35.Kurt,InvL350.Kurt,L35.Kurt,L350.Kurt,InvL35.WBC.s,InvL350.WBC.s,L35.WBC.s,L350.WBC.s
data_raw$ï..Sample,,,,,,,,,,,,,,,,,,,,,
HC201,0,17.482759,6.280992,22.415335,9.869444,8.933012,6.684726,9.641913,7.156108,9.287462,...,-1.178711,0.986264,4.323480,19.958736,5.498074,3.461117,1.552052,12.177875,1.451306,12.077231
HC201-2,0,18.807927,5.816456,21.627820,9.765512,9.206417,6.643071,9.498208,7.127413,9.352313,...,-0.581211,2.697019,5.469686,9.423986,4.613987,14.301870,1.259917,15.901688,2.614666,20.816450
HC202,0,17.379592,6.710419,21.210317,9.471429,8.893001,6.757038,9.380965,7.066555,9.136983,...,-0.622017,2.009628,4.220477,7.244893,4.289329,8.731232,1.158204,12.395936,1.199233,10.567577
HC203,0,17.895522,6.588986,19.823961,10.166667,8.998976,6.754368,9.106248,7.194440,9.052612,...,-0.572913,2.337535,5.055472,2.856551,3.955246,13.679129,2.172179,21.017737,2.365974,17.411341
HC204,0,18.218487,8.420266,20.050279,10.278816,9.062188,7.149129,9.129473,7.201170,9.095831,...,-0.197996,2.655057,4.816054,2.525604,5.409479,19.918003,1.310566,10.097907,1.020519,10.768864


In [17]:
for s in new_feature:
    feature_df.insert(loc = 1, column = s, value = df[s].values)

In [18]:
feature_df.to_csv('new MK data.csv')

### Question left:

0. the shape mismatches: 86 vs 83

1. the new feature for each kind of chip(inv35,350 etc), how do we combine them together? or just leave them as invidividual features?

2. the bimodal fitting doesn't work well. the bimodal characteristic can also be shown in the R2 value in unuimodal Gaussian thus can skip a direct bimodal fitting?




## Exploration on more combinations 

In [19]:
import pandas as pd
feature_df = pd.read_csv('new MK data.csv',header=[0], skiprows=0, index_col=0)

In [20]:
feature_df.columns.values

array(['Patient', 'R2_L350', 'R2_L35', 'R2_invL350', 'R2_invL35',
       'mode_mean_L350', 'mode_mean_L35', 'mode_mean_invL350',
       'mode_mean_invL35', 'kl_L350', 'kl_L35', 'kl_invL350', 'kl_invL35',
       'kl_L', 'kl_invL', 'InvL35.Mean', 'InvL350.Mean', 'L35.Mean',
       'L350.Mean', 'Size_InvL35', 'Size_InvL350', 'Size_L35',
       'Size_L350', 'MeanSize_35', 'MeanSize_350', 'MeanSize_InvL',
       'MeanSize_L', 'InvL35.SD', 'InvL350.SD', 'L35.SD', 'L350.SD',
       'InvL35.Skewness', 'InvL350.Skewness', 'L35.Skewness',
       'L350.Skewness', 'InvL35.Kurt', 'InvL350.Kurt', 'L35.Kurt',
       'L350.Kurt', 'InvL35.WBC.s', 'InvL350.WBC.s', 'L35.WBC.s',
       'L350.WBC.s'], dtype=object)

In [55]:
# count
feature_df['combo1'] = feature_df.apply (lambda row: row['InvL35.WBC.s']*row['InvL350.WBC.s']/row['L35.WBC.s']/row['L350.WBC.s'], axis=1)

In [56]:
# count
feature_df['combo2'] = feature_df.apply (lambda row: (row['L35.WBC.s']+row['L350.WBC.s'])/((row['InvL35.WBC.s']+row['InvL350.WBC.s'])), axis=1)

In [57]:
# closeness to standard
feature_df['combo3'] = feature_df.apply (lambda row: row['L35.Skewness']*row['L35.Kurt']*row['L35.SD']/row['R2_L35'], axis=1)

In [58]:
# closeness to standard
feature_df['combo4'] = feature_df.apply (lambda row: row['L35.Skewness']*row['kl_L35']*row['L35.Kurt']*row['mode_mean_L35']/row['R2_L35'], axis=1)

In [59]:
# size
feature_df['combo5'] = feature_df.apply (lambda row: row['Size_L35']*row['Size_InvL350']*row['Size_L350']*row['Size_InvL35'], axis=1)

In [60]:
# size
feature_df['combo6'] = feature_df.apply (lambda row: row['MeanSize_L']*row['MeanSize_35']/(row['L35.WBC.s'] + row['L350.WBC.s']), axis=1)

In [61]:
# difference
feature_df['combo7'] = feature_df.apply (lambda row: row['kl_L35']*row['kl_L350']*row['kl_L']*row['kl_invL'], axis=1)

In [62]:
# multi
feature_df['combo8'] = feature_df.apply (lambda row: row['L35.WBC.s']*row['Size_L35']*row['MeanSize_L']+row['kl_L35'], axis=1)

In [63]:
# multi
feature_df['combo9'] = feature_df.apply (lambda row: row['combo5']/row['combo4'], axis=1)

In [64]:
# multi
feature_df['combo10'] = feature_df.apply (lambda row:row['combo3']/row['combo6'], axis=1)

In [67]:
feature_df.to_csv('new MK data.csv')